In [11]:
# when installing the libraries after cloning the repository, use this command: pip install -r requirements.txt
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

import setuptools.dist

from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, LSTM
from keras.utils import pad_sequences

In [17]:
train_data = pd.read_csv("train.csv")